In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.stats as sps
import requests


In [ ]:
URL = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-07-07/coffee_ratings.csv'
response = requests.get(URL)
status_code = response.status_code

try:
    if status_code == 200:
        print("Ссылка получает ответ от сервера 200 - можно продолжать работу")
    else:
        print(f"Ссылка получает ответ от сервера {status_code} - убедитесь в интернет-соединении или правильности ссылки")
except requests.exceptions.RequestException as e:
    print(f"Ошибка при проверке ссылки: {e}")


In [ ]:
df = pd.read_csv(URL)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.fillna(0).isna().sum()

2. Какие страны являются крупными экспортерами кофе? 
3. Каковы корреляции между различными показателями оценки кофе?
4. Как (если есть) влияние цвета зерен на общий сорт кофе?
5. Влияет ли страна происхождения на качество кофе?
6. Существенно ли влияет высота на качество кофе?

2. Какие страны являются крупными экспортерами кофе? 

In [ ]:
most_popular_country = df['country_of_origin'].value_counts()
most_popular_country.head()

In [ ]:
sns.barplot(x=most_popular_country.index[most_popular_country > 20],
            y=most_popular_country.values[most_popular_country > 20])
plt.title('The biggest exporters of coffee')
plt.xticks(rotation=90)
plt.show()

3. Каковы корреляции между различными показателями оценки кофе?

К показателям оценки кофе относят: аромат, вкус, баланс, кислотность, однородность и тд.
Поэтому к построению корреляции пришлось использовать достаточно много столбцов.
Источник: https://spiritanimalcoffee.com/blogs/spirit-animal-blog/the-coffee-quality-score


In [ ]:
corr = df[['cupper_points', 'aroma', 'flavor', 'aftertaste', 'body', 'acidity', 'balance', 'uniformity', 'sweetness', 'clean_cup']].corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True);

По тепловой карте можно сделать вывод, что на оценку качества кофе в основном влияют вкусовые параметры:
    - аромат
    - вкус (наибольшее влияние)
    - послевкусие (наибольшее влияние)
    - баланс
    - однородность
    
Что странно, то сладость кофе - не является важным показателем для оценки кофе эскпертами, хотя и является вкусовой характеристикой
Более того, это является наименнее важным показателем.

4. Как (если есть) влияние цвета зерен на общий сорт кофе?

In [ ]:
df['color'].value_counts()

In [ ]:
df['species'].value_counts()

In [ ]:
pd.crosstab(df['species'], df['color'])

Благодаря кросс-таблице можно наглядно увидеть, что с большей долей вероятности для текущего дата сета цвет зерен Green присущ сорту Arabica.
Для Robusta сликом мало данных, однако результат будет такой же.

Если бы у нас не было данных по Robusta или зернам этого сорта было присуще большинство другого цвета можно было бы сделать вывод, что сорт действительно зависит от цвета, однако сейчас - это не так.


5. Влияет ли страна происхождения на качество кофе?

In [ ]:
specialty_df = df[df['total_cup_points'] >= 80]

In [ ]:
total_counts = df['country_of_origin'].value_counts()
filtered_counts = specialty_df['country_of_origin'].value_counts()
perc = (filtered_counts / total_counts) * 100
perc.sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(12, 4))
sns.barplot(x=perc.index[perc > 90],
            y=perc.values[perc > 90])
plt.xticks(rotation=90)
plt.show()

На графике только те страны, что экспортировали исключительно Specialty кофе ко всему экспорту, 100% сырья было total_cup_points >= 80

Из этого можно сделать вывод - страна происхождения влияет на качество кофе тк такие страны действительно есть.

6. Существенно ли влияет высота на качество кофе?

In [ ]:
specialty_df['altitude_mean_meters'].mean().round(2)

Как видим, что с среднем Specialty кофейные зерна произростают в среднем на высоте - 1661.2m

In [ ]:
not_specialty_df = df[df['total_cup_points'] < 80]
not_specialty_df['altitude_mean_meters'].mean().round(2)

А если брать обратную ситуацию, то зерна, которые имеют качество ниже произрастают значительно выше.

In [ ]:
Результаты моего анализа по этому заданию получились весьма протеворичивы тк на большинстве ресурсов указана как раз обратная информация. 
Что чем выще растут кофейные зерна - тем качественее напиток будет по итогу. 
Хотелось бы получить фидбек по этому заданию тк интересно, как правильно необходимо было проводить этот анализ